# Data Preprocessing
Studenten: Wietske Boersma, Isa van den Hoven, Alecsandru Kreefft-Libiu, Timber Rijsdorp

Teamnummer: N5

In [1]:
import pandas as pd

## Dataset 1: Kanker

Deze dataset bevat informatie over de hoeveelheid mensen die overlijden aan verschillende soorten kanker. De data laat per kankersoort zien wat de leeftijd gestandaardiseerde hoeveelheid mensen zijn die overleden zijn aan die kankersoort in dat jaar en in dat land. Wij zijn voor ons onderzoek alleen geinteresseerd in de data rondom longkanker omdat die kankersoort voornamelijk wordt benoemd in discussie rondom luchtvervuiling. 

Hieronder kan je zien hoe de data word't gefilterd. We selecteren alleen de kolommen: landcode, jaar en longkanker overledenen, en we verwijderen datapunten dat niet verbonden is aan een land.

In [2]:
# https://www.kaggle.com/datasets/programmerrdai/cancer?select=cancer-death-rates-by-type.csv
# Load data
df_cancer_rate = pd.read_csv('datasets/cancer-death-rates-by-type.csv')
# Select columns
df_cancer_rate = df_cancer_rate[['Code', 'Year', 'Deaths - Tracheal, bronchus, and lung cancer - Sex: Both - Age: Age-standardized (Rate)']]
# Rename columns
df_cancer_rate = df_cancer_rate.rename(columns={'Deaths - Tracheal, bronchus, and lung cancer - Sex: Both - Age: Age-standardized (Rate)':'TBL rate'})
# Delete non-country entries
df_cancer_rate = df_cancer_rate[df_cancer_rate['Code'].notna()]
# Display dataframe
df_cancer_rate.head(5)

,Code,Year,TBL rate
0,AFG,1990,14.069657
1,AFG,1991,13.805590
2,AFG,1992,13.591141
3,AFG,1993,13.422799
4,AFG,1994,13.250556


## Dataset 2: Luchtvervuiling

In deze dataset staat de hoeveelheid vervuiling voor verschillende stoffen dat is gemeten per land per jaar. We zijn geinteresseerd in alle soorten uitstoot voor dit onderzoek, dus de enige variabel dat we eruit filteren is het land omdat de landcode deze informatie al geeft. Verder verwijderen we weer datapunten dat niet verbonden is met een land.

In [3]:
# https://ourworldindata.org/explorers/air-pollution?tab=table&uniformYAxis=0&Pollutant=All+pollutants&Sector=From+all+sectors+%28Total%29&Per+capita=false&country=USA~CHN~IND~GBR~OWID_WRL
# Load data
df_pollution = pd.read_csv('datasets/air-pollution.csv')
# Select columns
df_pollution = df_pollution.drop('Entity', axis=1)
# Delete non-country entries
df_pollution = df_pollution[df_pollution['Code'].notna()]
# Display dataframe
df_pollution.head(5)

,Code,Year,Nitrogen oxide (NOx),Sulphur dioxide (SO₂) emissions,Carbon monoxide (CO) emissions,Organic carbon (OC) emissions,Non-methane volatile organic compounds (NMVOC) emissions,Black carbon (BC) emissions,Ammonia (NH₃) emissions
0,AFG,1750,555.47860,174.87167,142073.31,5456.8850,13596.633,1633.0308,7681.0464
1,AFG,1760,578.50757,181.99332,147859.23,5679.1167,14150.870,1699.5359,8000.8574
2,AFG,1770,602.47980,189.38850,153867.40,5909.8840,14726.470,1768.5956,8333.9610
3,AFG,1780,627.43220,197.06535,160104.42,6149.4410,15324.077,1840.2854,8680.9060
4,AFG,1790,653.40310,205.03189,166576.77,6398.0370,15944.332,1914.6805,9042.2660


## Dataset 3: Roken

Voor de laatste dataset hebben we gekeken naar het percentage van de mensen per land dat rookt. Deze dataset bevat ook per land per jaar het percentage dagelijkse rokers over de gehele populatie. We selecteren deze kolommen en verwijderen de niet-landelijke data. 

In [4]:
# https://ourworldindata.org/grapher/daily-smoking-prevalence-bounds?tab=chart
# Load data
df_smoking = pd.read_csv('datasets/smoking.csv')
# Select columns
df_smoking = df_smoking[['Code', 'Year', 'Daily smoking prevalence - both (IHME, GHDx (2012))']]
# Rename columns
df_smoking = df_smoking.rename(columns={'Daily smoking prevalence - both (IHME, GHDx (2012))':'Percentage daily smokers'})
# Delete non-country entries
df_smoking = df_smoking[df_smoking['Code'].notna()]
# Display dataframe
df_smoking.head(5)

,Code,Year,Percentage daily smokers
0,AFG,1980,10.4
1,AFG,1981,10.5
2,AFG,1982,10.5
3,AFG,1983,10.5
4,AFG,1984,10.6


In [5]:
df_countries = pd.read_csv('datasets/countries.csv')
df_countries = df_countries[['Country Code', 'year', 'Population, total', 'Surface area (sq. km)']]
df_countries = df_countries.rename(columns={'Country Code': 'Code', 'year':'Year', 'Population, total':'Population', 'Surface area (sq. km)':'Surface area'})
df_countries = df_countries[df_countries['Code'].notna()]
df_countries.head(5)

,Code,Year,Population,Surface area
0,AFG,1960,8996973.0,NaN
1,AFG,1961,9169410.0,652860.0
2,AFG,1962,9351441.0,652860.0
3,AFG,1963,9543205.0,652860.0
4,AFG,1964,9744781.0,652860.0


## Filtreren

Alle datasets bevatten jaarlijke data, maar niet alle datasets hebben dezelfde jaartallen. Om te zorgen dat we geen lege data krijgen in jaren dat een dataset niet heeft, filtreren we alle datasets naar alle jaren tussen 1990 en 2012. 

In [6]:
# Select years to collect data from
begin_year = 1990
end_year = 2012
df_cancer_year = df_cancer_rate[df_cancer_rate['Year'].between(begin_year, end_year)]
df_smoking_year = df_smoking[df_smoking['Year'].between(begin_year, end_year)]
df_pollution_year = df_pollution[df_pollution['Year'].between(begin_year, end_year)]
df_countries_year = df_countries[df_countries['Year'].between(begin_year, end_year)]

## Combineren
Nu gaan we de datasets samenvoegen. Dit doen we door de datasets te mergen op de landcode en jaar. Verder wordt nog de overige data dat uit landen komt die niet voorkomen in de rest van de datasets verwijdert.

In [7]:
# Merge dataframes on country code and year
df_combined = pd.merge(left=df_smoking_year, right=df_cancer_year, how='left', on=['Code', 'Year'])
df_combined = pd.merge(left=df_pollution_year, right=df_combined, how='left', on=['Code', 'Year'])
# Only select entries that are represented in all datasets
df_combined = df_combined[df_combined['TBL rate'].notna()]
df_combined = df_combined[df_combined['Percentage daily smokers'].notna()]

# Display data
df_combined.head(5)

,Code,Year,Nitrogen oxide (NOx),Sulphur dioxide (SO₂) emissions,Carbon monoxide (CO) emissions,Organic carbon (OC) emissions,Non-methane volatile organic compounds (NMVOC) emissions,Black carbon (BC) emissions,Ammonia (NH₃) emissions,Percentage daily smokers,TBL rate
0,AFG,1990,425144.75,12876.9610,1013430.94,28106.210,404866.40,8362.603,73274.35,10.8,14.069657
1,AFG,1991,413349.72,12671.9840,983752.06,28803.910,381666.22,8494.117,77547.38,10.9,13.805590
2,AFG,1992,272757.10,7732.8310,654986.94,29685.040,242334.94,8487.974,83017.66,11.0,13.591141
3,AFG,1993,276675.40,7967.0625,662752.90,30824.832,240105.75,8756.007,89469.49,11.1,13.422799
4,AFG,1994,252820.98,7698.9930,657333.00,32070.424,234383.25,9055.427,95695.22,11.2,13.250556


## Per duizend inwoners

In [8]:
columns = ['Nitrogen oxide (NOx)', 'Sulphur dioxide (SO₂) emissions', 'Carbon monoxide (CO) emissions', 'Organic carbon (OC) emissions', 'Non-methane volatile organic compounds (NMVOC) emissions', 'Black carbon (BC) emissions', 'Ammonia (NH₃) emissions']
df_norm = pd.merge(left=df_countries_year, right=df_combined, how='left', on=['Code', 'Year'])
df_norm = df_norm[df_norm['TBL rate'].notna()]
df_norm = df_norm[df_norm['Percentage daily smokers'].notna()]
for column in columns:
    df_norm[column] = (df_norm[column]/df_norm['Population']) * 1000
df_norm = df_norm.drop(['Population', 'Surface area'], axis=1)
df_norm.head(5)

,Code,Year,Nitrogen oxide (NOx),Sulphur dioxide (SO₂) emissions,Carbon monoxide (CO) emissions,Organic carbon (OC) emissions,Non-methane volatile organic compounds (NMVOC) emissions,Black carbon (BC) emissions,Ammonia (NH₃) emissions,Percentage daily smokers,TBL rate
0,AFG,1990,34.251869,1.037435,81.647260,2.264382,32.618140,0.673735,5.903362,10.8,14.069657
1,AFG,1991,31.081224,0.952851,73.971788,2.165868,28.698829,0.638703,5.831061,10.9,13.805590
2,AFG,1992,18.829604,0.533831,45.216586,2.049287,16.729431,0.585962,5.731069,11.0,13.591141
3,AFG,1993,17.492720,0.503715,41.902354,1.948891,15.180614,0.553596,5.656682,11.1,13.422799
4,AFG,1994,14.805869,0.450874,38.495169,1.878129,13.726107,0.530310,5.604167,11.2,13.250556


## Opslaan

En als laatst wordt de gecombineerde dataset opgeslagen zodat het makkelijk toegangbaar is voor het onderzoek.

In [9]:
# Save to csv
df_combined.to_csv('datasets/combined.csv')
df_norm.to_csv('datasets/norm.csv')

In [10]:
df_norm.corr(numeric_only=True)

,Year,Nitrogen oxide (NOx),Sulphur dioxide (SO₂) emissions,Carbon monoxide (CO) emissions,Organic carbon (OC) emissions,Non-methane volatile organic compounds (NMVOC) emissions,Black carbon (BC) emissions,Ammonia (NH₃) emissions,Percentage daily smokers,TBL rate
Year,1.000000,-0.038595,-0.104656,-0.127479,-0.043288,-0.055879,-0.036491,-0.022930,-0.113890,-0.033405
Nitrogen oxide (NOx),-0.038595,1.000000,0.557357,0.432135,-0.228779,0.514666,0.264043,0.189436,0.194805,0.393946
Sulphur dioxide (SO₂) emissions,-0.104656,0.557357,1.000000,0.168785,-0.158816,0.561966,0.239897,-0.001822,0.198109,0.254374
Carbon monoxide (CO) emissions,-0.127479,0.432135,0.168785,1.000000,0.097245,0.302241,0.216195,0.412569,0.059140,0.217940
Organic carbon (OC) emissions,-0.043288,-0.228779,-0.158816,0.097245,1.000000,-0.006287,0.514381,0.039748,-0.227585,-0.110677
Non-methane volatile organic compounds (NMVOC) emissions,-0.055879,0.514666,0.561966,0.302241,-0.006287,1.000000,0.491063,0.116205,-0.056746,0.079713
Black carbon (BC) emissions,-0.036491,0.264043,0.239897,0.216195,0.514381,0.491063,1.000000,0.015943,-0.136390,-0.016978
Ammonia (NH₃) emissions,-0.022930,0.189436,-0.001822,0.412569,0.039748,0.116205,0.015943,1.000000,0.111545,0.218620
Percentage daily smokers,-0.113890,0.194805,0.198109,0.059140,-0.227585,-0.056746,-0.136390,0.111545,1.000000,0.641703
TBL rate,-0.033405,0.393946,0.254374,0.217940,-0.110677,0.079713,-0.016978,0.218620,0.641703,1.000000
